In [ ]:
!pip install transformers
!pip install tokenizers
!pip install datasets
!pip install sentencepiece

In [ ]:
!pip install accelerate

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
wikiann_dataset = load_dataset("wikiann","id")

In [ ]:
from datasets import load_dataset
indonlu_dataset = load_dataset("indonlu","nergrit")

In [28]:
wikiann_label_names=wikiann_dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
wikiann_label_names

In [67]:
indonlu_label_names=indonlu_dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
indonlu_label_names

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cahya/distilbert-base-indonesian")

In [ ]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

In [41]:
label_all_tokens=True
def tokenize_and_aligning_labels(examples):
  tokenize_input=tokenizer(examples["tokens"],truncation=True,is_split_into_words=True)
  labels=[]
  all_labels=examples["ner_tags"]

  for i,label in enumerate(all_labels):
    word_id=tokenize_input.word_id(batch_index=1)
    previous_word_index=None
    label_id=[]
    for word_index in word_id:
      if word_index is None:
        label_id.append(--100)
      elif word_index!=previous_word_index:
        label_id.append(label[word_index])
      else:
        label_id.append(label[word_index]  if label_all_tokens else -100)
      previous_word_index=word_index             
    labels.append(label_id)
  tokenize_input["labels"]=labels
  return tokenize_input

In [43]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [47]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [48]:
tokenize_indonlu_dataset=indonlu_dataset.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [90]:
from transformers import DefaultDataCollator

data_collator=DefaultDataCollator(return_tensors="tf")

In [91]:
tokenize_indonlu_dataset["train"]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1672
})

In [81]:
from transformers import AutoModel
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tf_train_dataset = tokenize_indonlu_dataset["train"].to_tf_dataset(
  columns=['tokens','input_ids', 'attention_mask', 'labels'],
  shuffle=False,
  batch_size=16,
  collate_fn=data_collator,
)
validation_dataset=tokenize_indonlu_dataset["validation"].to_tf_dataset(
  columns=['tokens','input_ids', 'attention_mask', 'labels'],
  shuffle=False,
  batch_size=16,
  collate_fn=data_collator,
)

In [96]:
from transformers import DefaultDataCollator

pytorch_data_collator=DefaultDataCollator(return_tensors="pt")

In [109]:
from torch.utils.data import DataLoader

torch_train_dataset = DataLoader(tokenize_indonlu_dataset["train"],shuffle=True,collate_fn=pytorch_data_collator,
    batch_size=16,
)
torch_validation_dataset = DataLoader(
    tokenize_indonlu_dataset["validation"], collate_fn=pytorch_data_collator, batch_size=16
)

In [59]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16

#tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-squad", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
model.fit(tf_train_dataset, callbacks=[callback], epochs=num_train_epochs)

In [110]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [111]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, torch_train_dataset, torch_validation_dataset = accelerator.prepare(
    model, optimizer, torch_train_dataset, torch_validation_dataset
)

In [112]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(torch_train_dataset)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [99]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

'chanifrusydi/bert-finetuned-ner'

In [100]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

Cloning https://huggingface.co/chanifrusydi/bert-finetuned-ner into local empty directory.


In [101]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [115]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in torch_train_dataset:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in torch_validation_dataset:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/315 [00:00<?, ?it/s]

ValueError: ignored